
# Black Box in Flux — Minimal Pipeline

This notebook simulates a **driven–dissipative qubit** (your *quantum black box*) with a **slowly drifting parameter** (*flux*), records an observable as a black-box output, and computes both **classical chaos-style** diagnostics and a **quantum sensitivity** proxy.

**You get in one run:**
- PRBS drive + slow drift on detuning.
- Time series of ⟨σ_z⟩.
- Delay embedding, Rosenstein largest Lyapunov exponent (LLE), permutation entropy, recurrence rate.
- Uhlmann-fidelity divergence between two nearly identical evolutions (a Loschmidt-echo-like proxy).

> Tip: Start with the defaults, then change `DRIFT_RATE` (the flux) and compare metrics.


In [ ]:

import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import pdist, squareform
from scipy.signal import correlate
from math import log2
from qutip import basis, sigmax, sigmay, sigmaz, mesolve, Qobj, liouvillian, expect, qeye, Options, fidelity

np.random.seed(7)
plt.rcParams['figure.figsize'] = (7,4)


## 1) Helpers: PRBS, drift, simple metrics

In [ ]:

def prbs(length, switch_prob=0.02, low=-1.0, high=1.0):
    """Pseudo-random binary sequence with occasional flips."""
    x = np.zeros(length)
    state = high
    for t in range(length):
        if np.random.rand() < switch_prob:
            state = low if state == high else high
        x[t] = state
    return x

def slow_drift(length, rate=1e-4, start=0.0):
    """Linear slow drift; use as a small detuning change over time."""
    return start + rate * np.arange(length)

def delay_embed(x, m=3, tau=2):
    N = len(x) - (m-1)*tau
    if N <= 0:
        raise ValueError("Time series too short for embedding.")
    Y = np.zeros((N, m))
    for i in range(m):
        Y[:, i] = x[i*tau:i*tau+N]
    return Y

def permutation_entropy(x, order=3, delay=1):
    """Permutation entropy (Bandt & Pompe)."""
    n = len(x) - (order - 1) * delay
    if n <= 0:
        return np.nan
    patterns = {}
    for i in range(n):
        window = x[i:i+order*delay:delay]
        key = tuple(np.argsort(window))
        patterns[key] = patterns.get(key, 0) + 1
    probs = np.array(list(patterns.values()), dtype=float)
    probs /= probs.sum()
    pe = -np.sum(probs * np.log2(probs))
    return pe / log2(np.math.factorial(order))

def recurrence_rate(Y, eps=None):
    D = squareform(pdist(Y))
    if eps is None:
        # Median heuristic
        eps = np.median(D[D>0])
    R = (D < eps).astype(int)
    # Exclude trivial diagonal
    N = len(Y)
    return (np.sum(R) - N) / (N*N - N)

def rosenstein_lle(x, m=3, tau=2, mean_window=50, dt=1.0):
    """Simple Rosenstein LLE estimate on delay-embedded data."""
    Y = delay_embed(x, m=m, tau=tau)
    N = len(Y)
    # For each point, find nearest neighbor not too close in time (Theiler window)
    theiler = m * tau
    nn_idx = np.zeros(N, dtype=int)
    nn_dist = np.zeros(N)
    for i in range(N):
        d = np.linalg.norm(Y - Y[i], axis=1)
        d[max(0, i-theiler):min(N, i+theiler+1)] = np.inf
        j = np.argmin(d)
        nn_idx[i] = j
        nn_dist[i] = d[j]
    # Track divergence over k steps
    kmax = min(mean_window, N-1)
    div = np.zeros(kmax)
    count = np.zeros(kmax)
    for i in range(N - kmax):
        j = nn_idx[i]
        for k in range(kmax):
            di = np.linalg.norm(Y[i+k] - Y[j+k])
            if np.isfinite(di) and di>0:
                div[k] += np.log(di)
                count[k] += 1
    valid = count > 0
    t = np.arange(kmax)[valid] * dt
    y = (div[count>0] / count[count>0])
    if len(t) < 2:
        return np.nan, t, y
    # Linear fit on early part
    K = max(5, len(t)//3)
    slope = np.polyfit(t[:K], y[:K], 1)[0]
    return slope, t, y


## 2) Quantum model: driven–dissipative qubit with slow detuning drift

In [ ]:

# Simulation parameters
T_STEPS    = 5000       # total time steps
DT         = 0.01       # step duration (arb. units)
DRIFT_RATE = 2e-4       # drift per step in detuning (the "flux")
OMEGA0     = 1.0        # base Rabi drive strength
DELTA0     = 0.2        # base detuning
GAMMA      = 0.05       # relaxation rate (Lindblad)
SWITCH_P   = 0.01       # PRBS switch probability
OBS        = sigmaz()   # observable to record (black-box output)
 
# Build time-dependent pieces
u = prbs(T_STEPS, switch_prob=SWITCH_P, low=-1.0, high=1.0)      # drive modulation
drift = slow_drift(T_STEPS, rate=DRIFT_RATE, start=0.0)          # slow detuning drift
times = np.arange(T_STEPS) * DT

sx, sy, sz = sigmax(), sigmay(), sigmaz()
H0 = 0.5 * DELTA0 * sz
 
def H_t(t, args):
    idx = int(t/DT)
    if idx >= T_STEPS: idx = T_STEPS-1
    delta_t = DELTA0 + drift[idx]
    omega_t = OMEGA0 * (1 + 0.2 * u[idx])  # drive modulated by PRBS
    return 0.5 * delta_t * sz + 0.5 * omega_t * sx

c_ops = [np.sqrt(GAMMA) * (basis(2,0)*basis(2,1).dag())]  # |1> -> |0| relaxation
rho0 = (basis(2,0)*basis(2,0).dag())                      # start in ground
 
opts = Options(nsteps=2000, atol=1e-7, rtol=1e-6)
res = mesolve(H_t, rho0, times, c_ops=c_ops, e_ops=[OBS], options=opts)
z_t = np.real(res.expect[0])
 
plt.plot(times, z_t)
plt.xlabel("time")
plt.ylabel("<sigma_z>")
plt.title("Black-box output time series")
plt.show()


## 3) Classical chaos-style diagnostics on black-box output

In [ ]:

EMB_DIM = 5
TAU     = 2
 
Y = delay_embed(z_t, m=EMB_DIM, tau=TAU)

# Rosenstein LLE
lle, t_lle, y_lle = rosenstein_lle(z_t, m=EMB_DIM, tau=TAU, mean_window=60, dt=DT)
print("Estimated LLE (Rosenstein):", lle)

plt.plot(t_lle, y_lle)
plt.xlabel("time")
plt.ylabel("mean log divergence")
plt.title(f"Rosenstein plot (slope ≈ {lle:.4f})")
plt.show()

# Permutation entropy (normalized)
pe = permutation_entropy(z_t, order=4, delay=1)
print("Permutation Entropy (norm):", pe)

# Recurrence rate
rr = recurrence_rate(Y, eps=None)
print("Recurrence Rate:", rr)


## 4) Quantum sensitivity proxy (Uhlmann fidelity divergence)

In [ ]:

# We re-run two evolutions with a tiny detuning offset in one of them
DELTA_EPS = 1e-2  # small Hamiltonian perturbation
 
def H_t_eps(t, args):
    idx = int(t/DT)
    if idx >= T_STEPS: idx = T_STEPS-1
    delta_t = (DELTA0 + drift[idx]) + DELTA_EPS   # perturbed detuning
    omega_t = OMEGA0 * (1 + 0.2 * u[idx])
    return 0.5 * delta_t * sz + 0.5 * omega_t * sx
 
res0 = mesolve(H_t, rho0, times, c_ops=c_ops, e_ops=[], options=opts)
res1 = mesolve(H_t_eps, rho0, times, c_ops=c_ops, e_ops=[], options=opts)
 
# Compute Uhlmann fidelity F(rho0(t), rho1(t)); echo-like sensitivity E=1-F
F = np.array([fidelity(res0.states[i], res1.states[i]) for i in range(len(times))])
E = 1.0 - F
 
fig, ax = plt.subplots()
ax.plot(times, E)
ax.set_xlabel("time")
ax.set_ylabel("1 - Fidelity")
ax.set_title("Echo-like sensitivity to tiny Hamiltonian nudge")
plt.show()

# Summaries to compare with classical metrics
echo_mean = float(np.mean(E[int(0.2*len(E)):]))
echo_max  = float(np.max(E))
print("Echo sensitivity (mean over last 80%):", echo_mean)
print("Echo sensitivity (max):", echo_max)


## 5) One-figure summary for a few flux (drift) levels

In [ ]:

def run_with_drift(drift_rate):
    global drift
    drift = slow_drift(T_STEPS, rate=drift_rate, start=0.0)
    res = mesolve(H_t, rho0, times, c_ops=c_ops, e_ops=[OBS], options=opts)
    z = np.real(res.expect[0])
    # classical metrics
    pe = permutation_entropy(z, order=4, delay=1)
    lle, *_ = rosenstein_lle(z, m=EMB_DIM, tau=TAU, mean_window=60, dt=DT)
    Y = delay_embed(z, m=EMB_DIM, tau=TAU)
    rr = recurrence_rate(Y, eps=None)
    # echo proxy
    res0 = mesolve(H_t, rho0, times, c_ops=c_ops, e_ops=[], options=opts)
    res1 = mesolve(H_t_eps, rho0, times, c_ops=c_ops, e_ops=[], options=opts)
    F = np.array([fidelity(res0.states[i], res1.states[i]) for i in range(len(times))])
    E = 1.0 - F
    echo_mean = float(np.mean(E[int(0.2*len(E)):]))
    return pe, lle, rr, echo_mean

drifts = [0.0, 5e-5, 1e-4, 2e-4, 5e-4]
results = []
for d in drifts:
    pe, lle, rr, em = run_with_drift(d)
    results.append((d, pe, lle, rr, em))

import pandas as pd
df = pd.DataFrame(results, columns=["drift_rate", "perm_entropy", "lle", "recurrence_rate", "echo_mean"])
display(df)

# Plot
fig, ax = plt.subplots()
ax.plot(df["drift_rate"], df["perm_entropy"], marker="o", label="Permutation Entropy")
ax.plot(df["drift_rate"], df["lle"], marker="o", label="LLE (Rosenstein)")
ax.plot(df["drift_rate"], df["recurrence_rate"], marker="o", label="Recurrence Rate")
ax.plot(df["drift_rate"], df["echo_mean"], marker="o", label="Echo Mean (1-F)")
ax.set_xlabel("drift rate (flux)")
ax.set_ylabel("metric (arb.)")
ax.set_title("Flux vs metrics summary")
ax.legend()
plt.show()
